In [113]:
# -*- coding: utf-8 -*-
# Cell 1: 匯入基礎模組
from aclg.rules.split.split_ratio import split_by_ratio, SplitOrientation, split_by_ratio_grid
from aclg.rules.split.split_basic import split_horizontal, split_vertical
from aclg.rules.split.split_hold import split_hold
from aclg.rules.spacing import spacing_grid, spacing_vertical, spacing_horizontal
from aclg.post_processing.padding import add_padding, add_padding_advanced, add_padding_random_oneside, add_padding_based_on_alignment
from aclg.rules.symetric.symmetric_1 import split_symmetric_1_horizontal, split_symmetric_1_vertical
from aclg.rules.align import align_components, AlignmentMode
from aclg.dataclass.component import Component
import random
import math
import numpy as np
from typing import List, Dict, Any, Tuple

In [114]:
import os
from datetime import datetime
import yaml

def load_yaml_config(path='config.yaml'):
    """
    從指定的路徑載入 YAML 設定檔。

    Args:
        path (str): YAML 檔案的路徑。

    Returns:
        dict: 包含設定參數的字典；如果檔案不存在或解析失敗則回傳 None。
    """
    try:
        with open(path, 'r', encoding='utf-8') as f:
            # 使用 safe_load 更安全，避免執行任意程式碼
            return yaml.safe_load(f)
    except FileNotFoundError:
        print(f"錯誤：找不到設定檔 '{path}'。")
        return None
    except yaml.YAMLError as e:
        print(f"錯誤：解析 YAML 檔案 '{path}' 失敗: {e}")
        return None

In [115]:
import math
import numpy as np
from typing import List, Tuple, Set

# 替換掉您原有的 NetlistGenerator 類別
class NetlistGenerator:
    """
    根據論文方法產生 Netlist，並透過尋找連通分量來確保
    整個圖是完全連通的，不存在孤立的元件群。
    """
    def __init__(self,
                 pin_dist_alpha: float = 2.5,
                 min_pins_per_comp: int = 2,
                 max_pins_per_comp: int = 50,
                 edge_scale_param: float = 15.0,
                 edge_gamma_multiplier: float = 0.05,
                 max_edge_prob: float = 0.9):
        self.pin_alpha = pin_dist_alpha
        self.min_pins = min_pins_per_comp
        self.max_pins = max_pins_per_comp
        self.s = edge_scale_param
        self.gamma = edge_gamma_multiplier
        self.max_p = max_edge_prob

    def _generate_pins_for_components(self, components: List[Component]) -> List[List[Tuple[float, float]]]:
        """為所有元件產生引腳座標。"""
        all_pins = []
        num_pins_array = np.random.zipf(self.pin_alpha, len(components)) + self.min_pins
        
        for i, comp in enumerate(components):
            comp_pins = []
            num_pins = int(np.clip(num_pins_array[i], self.min_pins, self.max_pins))
            left, top = comp.get_topleft()
            right, bottom = comp.get_bottomright()
            
            for _ in range(num_pins):
                pin_x = random.uniform(left, right)
                pin_y = random.uniform(top, bottom)
                comp_pins.append((pin_x, pin_y))
            all_pins.append(comp_pins)
        return all_pins

    def _generate_probabilistic_edges(self, all_pins: List[List[Tuple[float, float]]]) -> List[Tuple[Tuple[float, float], Tuple[float, float]]]:
        """ [第一階段] 根據引腳距離機率性地產生邊。"""
        edges = []
        num_components = len(all_pins)
        # 建立 pin 到 component 索引的映射，以避免重複計算
        pin_to_comp_map = {pin: i for i, comp_pins in enumerate(all_pins) for pin in comp_pins}
        all_pin_coords = list(pin_to_comp_map.keys())
        
        # 遍歷所有 pin 的組合
        for i in range(len(all_pin_coords)):
            for j in range(i + 1, len(all_pin_coords)):
                p1 = all_pin_coords[i]
                p2 = all_pin_coords[j]
                
                # 如果兩個 pin 屬於同一個元件，則跳過
                if pin_to_comp_map[p1] == pin_to_comp_map[p2]:
                    continue

                l1_distance = abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])
                prob = min(self.gamma * math.exp(-l1_distance / self.s), self.max_p)
                if random.random() < prob:
                    edges.append((p1, p2))
        return edges

    def _ensure_single_connected_component(self, components: List[Component], all_pins: List[List[Tuple[float, float]]], edges: List[Tuple[Tuple[float, float], Tuple[float, float]]]):
        """ [第二階段] 使用圖遍歷演算法確保只有一個連通分量。"""
        num_components = len(components)
        if num_components < 2: return

        # 1. 建立鄰接表和 pin -> component 的映射
        adj = {i: set() for i in range(num_components)}
        pin_to_comp_map = {pin: i for i, comp_pins in enumerate(all_pins) for pin in comp_pins}
        for p1, p2 in edges:
            comp_idx1 = pin_to_comp_map.get(p1)
            comp_idx2 = pin_to_comp_map.get(p2)
            if comp_idx1 is not None and comp_idx2 is not None:
                adj[comp_idx1].add(comp_idx2)
                adj[comp_idx2].add(comp_idx1)

        # 2. 尋找所有連通分量
        visited = set()
        components_groups = []
        for i in range(num_components):
            if i not in visited:
                group = []
                q = [i]
                visited.add(i)
                while q:
                    u = q.pop(0)
                    group.append(u)
                    for v in adj[u]:
                        if v not in visited:
                            visited.add(v)
                            q.append(v)
                components_groups.append(group)
        
        if len(components_groups) <= 1:
            print("[*] 圖已完全連通，無需修正。")
            return

        # 3. 如果存在多個群體，則建立橋接邊
        print(f"[*] 發現 {len(components_groups)} 個獨立的元件群，開始橋接...")
        main_group = components_groups[0]
        for i in range(1, len(components_groups)):
            group_to_bridge = components_groups[i]
            
            # 尋找連接 main_group 和當前 group 的最近的一對 pin
            min_dist = float('inf')
            best_bridge_edge = None
            
            for u_idx in main_group:
                for v_idx in group_to_bridge:
                    for p1 in all_pins[u_idx]:
                        for p2 in all_pins[v_idx]:
                            dist = math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
                            if dist < min_dist:
                                min_dist = dist
                                best_bridge_edge = (p1, p2)
            
            if best_bridge_edge:
                edges.append(best_bridge_edge)
                # 將橋接後的群體成員加入 main_group，以便後續連接
                main_group.extend(group_to_bridge)

    def generate(self, components: List[Component]) -> Tuple[List[List[Tuple[float, float]]], List[Tuple[Tuple[float, float], Tuple[float, float]]]]:
        """
        [公開] 產生 pins 和 edges，並確保圖是單一連通分量。
        """
        if not components: return [], []
        print(f"[*] 開始為 {len(components)} 個元件產生 Netlist...")
        
        all_pins = self._generate_pins_for_components(components)
        edges = self._generate_probabilistic_edges(all_pins)
        print(f"[*] 初始機率性產生了 {len(edges)} 條邊。")

        self._ensure_single_connected_component(components, all_pins, edges)
        
        print(f"[*] Netlist 產生完畢，最終總共有 {len(edges)} 條邊。")
        return all_pins, edges

In [116]:
import matplotlib.pyplot as plt
from typing import List, Tuple, Set
from aclg.dataclass.component import Component

# 請使用這個更新版的 ComponentPlotter 來確保 GapFiller 元件能被繪製
class ComponentPlotter:
    """
    視覺化工具，可以繪製元件、邊(edges)，以及僅繪製被連接的引腳(pins)。
    新版支援繪製非階層性新增的元件 (如 GapFiller)。
    """
    def _draw_recursive(self, ax, component: Component):
        # (此函式不變)
        top_left_x, top_left_y = component.get_topleft()
        width, height, level = component.width, component.height, component.level
        # 擴充顏色列表以支援更多層級，確保 Level 4 有獨特顏色
        LEVEL_COLORS = ['#FFB3BA', '#FFDFBA', '#FFFFBA', '#BAFFC9', '#BAE1FF', '#E0BBE4', '#FFD1DC', '#B2DFDB']
        color = LEVEL_COLORS[level % len(LEVEL_COLORS)]
        rect = plt.Rectangle((top_left_x, top_left_y), width, height,
                             linewidth=1.2, edgecolor='black', facecolor=color, alpha=0.8)
        ax.add_patch(rect)
        label = f"L{level}\nID:{component.relation_id}"
        ax.text(component.x, component.y, label, ha='center', va='center', fontsize=8, color='black')
        if component.sub_components:
            for sub_comp in component.sub_components:
                self._draw_recursive(ax, sub_comp)

    def _draw_netlist(self, ax, edges: List[Tuple[Tuple[float, float], Tuple[float, float]]]):
        # (此函式不變)
        if not edges:
            return

        connected_pins = set()
        for p1, p2 in edges:
            connected_pins.add(p1)
            connected_pins.add(p2)
        
        print(f"[*] 正在繪製 {len(edges)} 條邊...")
        for p1, p2 in edges:
            ax.plot([p1[0], p2[0]], [p1[1], p2[1]], color='#555555', linestyle='-', linewidth=0.7, alpha=0.6)
            
        print(f"[*] 正在繪製 {len(connected_pins)} 個已連接的引腳...")
        for px, py in connected_pins:
            ax.plot(px, py, 'o', color='black', markersize=2.5, alpha=0.8)

    def plot(self, components_to_plot: List[Component], title: str = "Component Layout",
             edges: List[Tuple[Tuple[float, float], Tuple[float, float]]] = None,
             output_filename: str = "component_visualization.png"): # << 修改點
        """
        繪製元件和 Netlist (邊與已連接的引腳)。
        """
        plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 
        plt.rcParams['axes.unicode_minus'] = False
        fig, ax = plt.subplots(1, figsize=(14, 14))
        
        if not components_to_plot:
            ax.set_title("元件列表為空")
            plt.show()
            return

        for comp in components_to_plot:
            self._draw_recursive(ax, comp)
        
        if edges:
            self._draw_netlist(ax, edges)
        
        ax.autoscale_view()
        ax.set_aspect('equal', adjustable='box')
        plt.title(title, fontsize=16)
        plt.xlabel("X-axis")
        plt.ylabel("Y-axis")
        plt.grid(True, linestyle='--', alpha=0.5)
        
        # << 修改點 >> 使用傳入的 output_filename 參數
        plt.savefig(output_filename, dpi=150)
        print(f"✅ 繪圖完成！圖片已儲存至 {output_filename}")
        # 在批次產生時，我們通常不希望立即顯示圖片，因此將 plt.show() 註解掉
        # plt.show() 
        plt.close(fig) # 畫完後關閉圖形，釋放記憶體，非常重要！

In [117]:
# class Level_0:
#     def __init__(self):
#         self.x = 0
#         self.y = 0
#         self.w_range = (100, 120)
#         self.h_range = (100, 120)
#         self.generate_rule = 'root'
#         self.level = 0
#         self.relation_id = 0
    
#     def generate(self) -> List[Component]:
#         return [Component(
#             x=self.x,
#             y=self.y,
#             width=random.randint(*self.w_range),
#             height=random.randint(*self.h_range),
#             relation_id=self.relation_id,
#             generate_rule=self.generate_rule,
#             level=self.level
#         )]

# 更新後的版本
class Level_0:
    def __init__(self, w_range=(100, 120), h_range=(100, 120)):
        self.x = 0
        self.y = 0
        self.w_range = tuple(w_range) # 從 config 讀取的是 list，轉為 tuple
        self.h_range = tuple(h_range)
        self.generate_rule = 'root'
        self.level = 0
        self.relation_id = 0
    
    def generate(self) -> List[Component]:
        return [Component(
            x=self.x,
            y=self.y,
            width=random.randint(*self.w_range),
            height=random.randint(*self.h_range),
            relation_id=self.relation_id,
            generate_rule=self.generate_rule,
            level=self.level
        )]

In [118]:
class Level_1:
    """
    進階版的 Level_1 處理器，修正了對齊邏輯，嚴格遵守分割方向與對齊模式的綁定關係。
    """
    def __init__(
        self,
        w_h_ratio_bound: tuple[float, float] = (1/6, 6/1),
        max_tries_per_orientation: int = 50,
        num_splits_range: tuple[int, int] = (2, 5),
        ratio_range: tuple[float, float] = (0.3, 1.0),
        split_only_probability: float = 0.5,
        align_scale_factor_range: tuple[float, float] = (0.2, 1.0),
        force_align_threshold: int = 3
    ):
        # 儲存所有超參數
        self.w_h_ratio_bound = w_h_ratio_bound
        self.max_tries_per_orientation = max_tries_per_orientation
        self.num_splits_range = num_splits_range
        self.ratio_range = ratio_range
        self.split_only_probability = split_only_probability
        self.align_scale_factor_range = align_scale_factor_range
        self.force_align_threshold = force_align_threshold
        self.level = 1

    # _find_valid_ratios 輔助函式維持不變
    def _find_valid_ratios(self, parent_component: Component, orientation: SplitOrientation, num_splits: int):
        parent_w_h_ratio = parent_component.w_h_ratio()
        min_ratio, max_ratio = self.w_h_ratio_bound
        for _ in range(self.max_tries_per_orientation):
            ratios = [random.uniform(*self.ratio_range) for _ in range(num_splits)]
            total_ratio = sum(ratios)
            all_valid = True
            for r in ratios:
                sub_w_h_ratio = 0
                if orientation == SplitOrientation.HORIZONTAL:
                    sub_w_h_ratio = parent_w_h_ratio * (total_ratio / r)
                else:
                    sub_w_h_ratio = parent_w_h_ratio * (r / total_ratio)
                if not (min_ratio <= sub_w_h_ratio <= max_ratio):
                    all_valid = False
                    break
            if all_valid:
                return ratios
        return None

    def _apply_split(self, parent_component: Component, num_splits: int) -> List[Component]:
        """[行為1] 執行純分割操作"""
        if parent_component.w_h_ratio() > 1:
            orientations_to_try = [SplitOrientation.VERTICAL, SplitOrientation.HORIZONTAL]
        else:
            orientations_to_try = [SplitOrientation.HORIZONTAL, SplitOrientation.VERTICAL]

        for orientation in orientations_to_try:
            valid_ratios = self._find_valid_ratios(parent_component, orientation, num_splits)
            if valid_ratios:
                return split_by_ratio(parent_component, valid_ratios, orientation)

        return split_hold(parent_component)

    def _apply_align(self, parent_component: Component, num_splits: int) -> List[Component]:
        """[行為2] 執行分割後對齊操作 (策略二：使用數學限制法確保縮放合規)"""
        # 1. 隨機選擇對齊模式
        align_mode = random.choice(list(AlignmentMode))
        
        # 2. 根據您修正後的規則，決定分割方向
        if align_mode in [AlignmentMode.TOP, AlignmentMode.BOTTOM, AlignmentMode.CENTER_H]:
            required_orientation = SplitOrientation.VERTICAL
        else:
            required_orientation = SplitOrientation.HORIZONTAL

        # 3. 初始分割
        valid_ratios = self._find_valid_ratios(parent_component, required_orientation, num_splits)
        if not valid_ratios:
            return split_hold(parent_component)
        
        sub_components = split_by_ratio(parent_component, valid_ratios, required_orientation)

        # 4. 為每個子元件計算有效的縮放範圍，並從中生成縮放因子
        scale_factors = []
        min_ratio_bound, max_ratio_bound = self.w_h_ratio_bound
        min_scale_bound, max_scale_bound = self.align_scale_factor_range

        for comp in sub_components:
            original_ratio = comp.w_h_ratio()
            
            if align_mode in [AlignmentMode.TOP, AlignmentMode.BOTTOM, AlignmentMode.CENTER_H]:
                # 改變 height，計算 scale 的有效數學邊界
                valid_min_s = original_ratio / max_ratio_bound
                valid_max_s = original_ratio / min_ratio_bound
            else:
                # 改變 width，計算 scale 的有效數學邊界
                valid_min_s = min_ratio_bound / original_ratio
                valid_max_s = max_ratio_bound / original_ratio

            # 取【數學邊界】和【超參數邊界】的交集，確保縮放不會太誇張
            final_min_s = max(valid_min_s, min_scale_bound)
            final_max_s = min(valid_max_s, max_scale_bound)
            
            # 如果有效範圍不存在，則使用一個安全的預設值
            if final_min_s > final_max_s:
                scale = 1.0 
            else:
                scale = random.uniform(final_min_s, final_max_s)
            
            scale_factors.append(scale)

        # 5. 執行對齊
        return align_components(sub_components, scale_factors, align_mode)

    def _process_single_component(self, parent_component: Component) -> List[Component]:
        """[調度中心] 根據規則決策，並呼叫對應的處理函式"""
        num_splits = random.randint(*self.num_splits_range)
        
        # 1. 優先規則：檢查是否達到強制對齊的門檻
        if num_splits > self.force_align_threshold:
            # print(f"-> 規則觸發：元件數量 {num_splits} > {self.force_align_threshold}，強制對齊。")
            return self._apply_align(parent_component, num_splits)
        
        # 2. 降級規則：如果未達到門檻，則走原本的隨機機率決策
        else:
            if random.random() < self.split_only_probability:
                # print(f"-> 隨機決策：純分割")
                return self._apply_split(parent_component, num_splits)
            else:
                # print(f"-> 隨機決策：分割後對齊")
                return self._apply_align(parent_component, num_splits)

    def generate(self, components: List[Component]) -> List[Component]:
        """[公開方法] 處理元件列表"""
        all_results = []
        relation_id = 0
        for component in components:
            processed_sub_components = self._process_single_component(component)
            for sub_comp in processed_sub_components:
                sub_comp.level = self.level
                sub_comp.relation_id = relation_id
            all_results.extend(processed_sub_components)
            relation_id += 1
            component.sub_components = processed_sub_components
        return all_results

In [119]:
class Level_2:
    """
    Level 2 產生器 (上下文感知與多樣化策略版)。

    新版特性：
    1.  **多樣化切割**：除了原有的網格分割，新增了更簡單的線性分割選項，以增加版面變化性。
    2.  **機率性維持**：對於較大的元件，引入一個機率決定是否不進行切割，讓大型區塊得以保留。
    3.  **上下文感知**：(保留) 智慧對齊功能，會分析元件在同級中的相對位置。
    4.  **保留核心規則**：(保留) 單次對齊原則、允許間隙等先前版本的核心特性維持不變。
    """
    def __init__(
        self,
        # --- 原有參數 ---
        large_component_align_probability: float = 1.0,
        wide_threshold: float = 2.0,
        tall_threshold: float = 0.5,
        size_thresholds: Tuple[float, float] = (0.1, 0.4),
        small_component_hold_probability: float = 0.8,
        policy_wide: Dict[str, Any] = None,
        policy_tall: Dict[str, Any] = None,
        policy_square: Dict[str, Any] = None,
        w_h_ratio_bound: tuple[float, float] = (1/6, 6/1),
        max_tries: int = 50,
        ratio_grid_probability: float = 0.5,
        ratio_range: tuple[float, float] = (0.3, 0.6),
        # --- NEW: 新增用於增加多樣性的參數 ---
        large_component_hold_probability: float = 0.7, # NEW: 讓大元件維持原樣的機率
        simple_split_probability: float = 0.9,         # NEW: 使用簡單線性切割的機率
        num_splits_range: tuple[int, int] = (2, 4)     # NEW: 線性切割的數量範圍
    ):
        # __init__ 內容與之前版本相同
        self.large_component_align_probability = large_component_align_probability
        self.wide_threshold = wide_threshold
        self.tall_threshold = tall_threshold
        self.size_thresholds = size_thresholds
        self.small_component_hold_probability = small_component_hold_probability
        self.policy_wide = policy_wide or {"rows_range": (1, 2), "cols_range": (3, 5),"h_ratios_num_range": (1, 2), "v_ratios_num_range": (3, 5)}
        self.policy_tall = policy_tall or {"rows_range": (3, 5), "cols_range": (1, 2),"h_ratios_num_range": (3, 5), "v_ratios_num_range": (1, 2)}
        self.policy_square = policy_square or {"rows_range": (2, 4), "cols_range": (2, 4),"h_ratios_num_range": (2, 4), "v_ratios_num_range": (2, 4)}
        self.w_h_ratio_bound = w_h_ratio_bound
        self.max_tries = max_tries
        self.ratio_grid_probability = ratio_grid_probability
        self.ratio_range = ratio_range
        self.level = 2

        # --- NEW: 儲存新增的超參數 ---
        self.large_component_hold_probability = large_component_hold_probability
        self.simple_split_probability = simple_split_probability
        self.num_splits_range = num_splits_range


    # --- COPIED FROM Level_1: 用於實現簡單線性切割的輔助函式 ---
    def _find_valid_ratios(self, parent_component: Component, orientation: SplitOrientation, num_splits: int):
        parent_w_h_ratio = parent_component.w_h_ratio()
        min_ratio, max_ratio = self.w_h_ratio_bound
        for _ in range(self.max_tries): # 使用 max_tries 替代 max_tries_per_orientation
            ratios = [random.uniform(0.3, 1.0) for _ in range(num_splits)] # 使用固定範圍
            total_ratio = sum(ratios)
            all_valid = True
            for r in ratios:
                sub_w_h_ratio = 0
                if orientation == SplitOrientation.HORIZONTAL:
                    sub_w_h_ratio = parent_w_h_ratio * (total_ratio / r)
                else:
                    sub_w_h_ratio = parent_w_h_ratio * (r / total_ratio)
                if not (min_ratio <= sub_w_h_ratio <= max_ratio):
                    all_valid = False
                    break
            if all_valid:
                return ratios
        return None

    # --- NEW: 從 Level_1 借來的方法，用於執行「只切幾刀」的簡單分割 ---
    def _apply_simple_split(self, parent_component: Component) -> List[Component]:
        """[新行為] 執行簡單的線性分割（水平或垂直）。"""
        num_splits = random.randint(*self.num_splits_range)
        
        if parent_component.w_h_ratio() > 1:
            orientations_to_try = [SplitOrientation.VERTICAL, SplitOrientation.HORIZONTAL]
        else:
            orientations_to_try = [SplitOrientation.HORIZONTAL, SplitOrientation.VERTICAL]

        for orientation in orientations_to_try:
            valid_ratios = self._find_valid_ratios(parent_component, orientation, num_splits)
            if valid_ratios:
                return split_by_ratio(parent_component, valid_ratios, orientation)

        return split_hold(parent_component)

    # _apply_advanced_align 方法維持不變
    def _apply_advanced_align(self, parent_component: Component, siblings_bbox: Dict[str, float]) -> List[Component]:
        """[智慧規則] 根據元件在同級中的位置，過濾並選擇合適的對齊模式。"""
        valid_align_modes = []; epsilon = 1e-6
        p_left, p_top = parent_component.get_topleft()
        p_right, p_bottom = parent_component.get_bottomright()
        is_top_edge = abs(p_top - siblings_bbox['min_y']) < epsilon
        is_bottom_edge = abs(p_bottom - siblings_bbox['max_y']) < epsilon
        is_left_edge = abs(p_left - siblings_bbox['min_x']) < epsilon
        is_right_edge = abs(p_right - siblings_bbox['max_x']) < epsilon
        if is_top_edge: valid_align_modes.append(AlignmentMode.BOTTOM)
        if is_bottom_edge: valid_align_modes.append(AlignmentMode.TOP)
        if is_left_edge: valid_align_modes.append(AlignmentMode.RIGHT)
        if is_right_edge: valid_align_modes.append(AlignmentMode.LEFT)
        if not (is_top_edge or is_bottom_edge): valid_align_modes.append(AlignmentMode.CENTER_V)
        if not (is_left_edge or is_right_edge): valid_align_modes.append(AlignmentMode.CENTER_H)
        if not valid_align_modes:
            align_mode = AlignmentMode.CENTER_H if parent_component.w_h_ratio() <= 1 else AlignmentMode.CENTER_V
        else:
            align_mode = random.choice(valid_align_modes)
        if align_mode in [AlignmentMode.TOP, AlignmentMode.BOTTOM, AlignmentMode.CENTER_H]:
            orientation = SplitOrientation.VERTICAL
        else:
            orientation = SplitOrientation.HORIZONTAL
        num_splits = random.randint(2, 4)
        valid_ratios = [random.uniform(0.5, 1.0) for _ in range(num_splits)]
        sub_components = split_by_ratio(parent_component, valid_ratios, orientation)
        scale_factors = [random.uniform(0.6, 0.95) for _ in range(num_splits)]
        return align_components(sub_components, scale_factors, align_mode)

    # _apply_grid_split 方法維持不變
    def _apply_grid_split(self, parent_component: Component, size_ratio: float) -> List[Component]:
        """[標準規則] 為未被選中執行對齊的元件，執行常規的網格分割。"""
        shape_ratio = parent_component.w_h_ratio()
        base_policy = self.policy_square
        if shape_ratio > self.wide_threshold: base_policy = self.policy_wide
        elif shape_ratio < self.tall_threshold: base_policy = self.policy_tall
        final_policy = self._get_dynamic_policy(base_policy, size_ratio)
        if random.random() < self.ratio_grid_probability:
            return self._apply_ratio_grid(parent_component, final_policy)
        else:
            return self._apply_spacing_grid(parent_component, final_policy)

    # --- MODIFIED: generate 方法被重構以整合新的決策邏輯 ---
    def generate(self, components: List[Component], root_component: Component) -> List[Component]:
        """[公開方法] 處理整個元件列表，並確保最多只有一個元件被進階對齊。"""
        if not components or not root_component:
            return []
        
        root_area = root_component.width * root_component.height
        
        min_x = min(c.get_topleft()[0] for c in components)
        max_x = max(c.get_bottomright()[0] for c in components)
        min_y = min(c.get_topleft()[1] for c in components)
        max_y = max(c.get_bottomright()[1] for c in components)
        siblings_bbox = {"min_x": min_x, "max_x": max_x, "min_y": min_y, "max_y": max_y}
        
        small_thresh, large_thresh = self.size_thresholds
        alignment_candidates = [c for c in components if (c.width * c.height) / root_area > large_thresh]
        component_to_align = None
        if alignment_candidates and random.random() < self.large_component_align_probability:
            component_to_align = random.choice(alignment_candidates)

        all_results = []
        relation_id = 0
        for comp in components:
            processed_sub_components = []
            size_ratio = (comp.width * comp.height) / root_area

            # --- MODIFIED: 新的決策流程 ---
            if comp is component_to_align:
                # 1. 優先處理被選中進行「智慧對齊」的元件
                processed_sub_components = self._apply_advanced_align(comp, siblings_bbox)
            else:
                # 2. 處理其他普通元件
                is_large = size_ratio > large_thresh
                is_small = size_ratio < small_thresh

                if is_large and random.random() < self.large_component_hold_probability:
                    # 2a. 如果是大元件，根據機率決定是否維持原樣
                    processed_sub_components = split_hold(comp)
                elif is_small and random.random() < self.small_component_hold_probability:
                    # 2b. 如果是小元件，根據機率決定是否維持原樣
                    processed_sub_components = split_hold(comp)
                else:
                    # 2c. 對於中等元件，或通過了機率考驗的大元件，決定切割策略
                    if random.random() < self.simple_split_probability:
                        # 使用簡單線性切割
                        processed_sub_components = self._apply_simple_split(comp)
                    else:
                        # 使用複雜網格切割
                        processed_sub_components = self._apply_grid_split(comp, size_ratio)

            for sub_comp in processed_sub_components:
                sub_comp.level = self.level
                sub_comp.relation_id = relation_id
            all_results.extend(processed_sub_components)
            relation_id += 1
            comp.sub_components = processed_sub_components
            
        return all_results

    # (其他輔助函式與之前版本相同，此處省略)
    def _get_dynamic_policy(self, base_policy: Dict[str, Any], size_ratio: float) -> Dict[str, Any]:
        small_thresh, large_thresh = self.size_thresholds; dynamic_policy = base_policy.copy();
        if size_ratio < small_thresh: scale_factor = 0.5 
        elif size_ratio > large_thresh: scale_factor = 1.5 
        else: return dynamic_policy
        for key in ["rows_range", "cols_range", "h_ratios_num_range", "v_ratios_num_range"]:
            min_val, max_val = dynamic_policy[key]; new_min = max(1, int(min_val * scale_factor)); new_max = max(new_min, int(max_val * scale_factor)); dynamic_policy[key] = (new_min, new_max)
        return dynamic_policy
    def _apply_ratio_grid(self, parent_component: Component, policy: Dict[str, Any]) -> List[Component]:
        h_ratios_num_range = policy["h_ratios_num_range"]; v_ratios_num_range = policy["v_ratios_num_range"]
        for _ in range(self.max_tries):
            num_h = random.randint(*h_ratios_num_range); num_v = random.randint(*v_ratios_num_range)
            h_ratios = [random.uniform(*self.ratio_range) for _ in range(num_h)]; v_ratios = [random.uniform(*self.ratio_range) for _ in range(num_v)]
            return split_by_ratio_grid(parent_component, h_ratios, v_ratios)
        return split_hold(parent_component)
    def _apply_spacing_grid(self, parent_component: Component, policy: Dict[str, Any]) -> List[Component]:
        rows_range = policy["rows_range"]; cols_range = policy["cols_range"]
        for _ in range(self.max_tries):
            rows = random.randint(*rows_range); cols = random.randint(*cols_range)
            return spacing_grid(parent_component, rows, cols)
        return split_hold(parent_component)

In [120]:
import random
from typing import List
from aclg.dataclass.component import Component

class GapFiller:
    """
    Finds the longest available edge in a layout and places a row of 
    small, aligned components along it.
    """
    def __init__(self,
                 small_comp_w_range: tuple[float, float] = (6, 14),
                 small_comp_h_range: tuple[float, float] = (6, 14),
                 spacing: float = 0.5):
        """
        Initializes the GapFiller.
        Args:
            small_comp_w_range (tuple): Width range for new components.
            small_comp_h_range (tuple): Height range for new components.
            spacing (float): The gap to leave between newly placed components.
        """
        self.w_range = small_comp_w_range
        self.h_range = small_comp_h_range
        self.spacing = spacing
        self.level = 4

    def _check_collision(self, new_comp: Component, all_components: List[Component], root_component: Component) -> bool:
        # (此輔助函式與前一版完全相同)
        root_left, root_top = root_component.get_topleft()
        root_right, root_bottom = root_component.get_bottomright()
        new_left, new_top = new_comp.get_topleft()
        new_right, new_bottom = new_comp.get_bottomright()

        if not (new_left >= root_left and new_right <= root_right and new_top >= root_top and new_bottom <= root_bottom):
            return True

        for comp in all_components:
            comp_left, comp_top = comp.get_topleft()
            comp_right, comp_bottom = comp.get_bottomright()
            if (new_left < comp_right and new_right > comp_left and
                new_top < comp_bottom and new_bottom > comp_top):
                return True
        return False

    def fill(self, existing_leaf_components: List[Component], root_component: Component, num_to_place: int) -> List[Component]:
        """
        [主要方法] 實現尋找最佳邊緣並沿其線性排列的邏輯。
        """
        if not existing_leaf_components or num_to_place == 0:
            return []

        # 1. 尋找擁有最長邊的 host 元件
        best_host = None
        best_edge_type = ''
        max_edge_len = -1.0
        
        for comp in existing_leaf_components:
            if comp.width > max_edge_len:
                max_edge_len = comp.width
                best_host = comp
                best_edge_type = random.choice(['top', 'bottom'])
            if comp.height > max_edge_len:
                max_edge_len = comp.height
                best_host = comp
                best_edge_type = random.choice(['left', 'right'])

        if not best_host:
            return []

        # 2. 沿著找到的最佳邊緣，線性排列新元件
        gap_components = []
        all_components = existing_leaf_components.copy()
        h_left, h_top = best_host.get_topleft()
        h_right, h_bottom = best_host.get_bottomright()

        # 根據邊緣類型，初始化起始游標 (cursor)
        cursor = 0
        if best_edge_type in ['bottom', 'top']:
            cursor = h_left
        elif best_edge_type in ['left', 'right']:
            cursor = h_top

        for _ in range(num_to_place):
            new_w = random.uniform(*self.w_range)
            new_h = random.uniform(*self.h_range)
            new_comp = Component(x=0, y=0, width=new_w, height=new_h, level=self.level, relation_id=-1)
            
            # 根據邊緣類型，設定新元件位置並檢查邊界
            if best_edge_type == 'bottom':
                if cursor + new_w > h_right: break # 超出邊緣長度
                new_comp.x = cursor + new_w / 2
                new_comp.y = h_bottom + new_h / 2
            elif best_edge_type == 'top':
                if cursor + new_w > h_right: break
                new_comp.x = cursor + new_w / 2
                new_comp.y = h_top - new_h / 2
            elif best_edge_type == 'right':
                if cursor + new_h > h_bottom: break
                new_comp.x = h_right + new_w / 2
                new_comp.y = cursor + new_h / 2
            elif best_edge_type == 'left':
                if cursor + new_h > h_bottom: break
                new_comp.x = h_left - new_w / 2
                new_comp.y = cursor + new_h / 2

            # 進行碰撞檢測
            if not self._check_collision(new_comp, all_components, root_component):
                gap_components.append(new_comp)
                all_components.append(new_comp)
                # 移動游標，準備放下一塊
                if best_edge_type in ['top', 'bottom']:
                    cursor += new_w + self.spacing
                else:
                    cursor += new_h + self.spacing
            else:
                # 如果路徑被阻擋，就停止放置
                break
        
        return gap_components

In [121]:
# --- NEW: JSON Export Functionality ---
import json
from typing import Any

def component_to_dict(component: Component) -> Dict[str, Any]:
    """
    遞迴地將一個 Component 物件及其所有子元件轉換成字典格式。
    """
    if component is None:
        return None
    
    sub_components_list = []
    if component.sub_components:
        sub_components_list = [component_to_dict(sub) for sub in component.sub_components]

    return {
        "x": component.x,
        "y": component.y,
        "width": component.width,
        "height": component.height,
        "level": component.level,
        "relation_id": component.relation_id,
        "generate_rule": component.generate_rule,
        "sub_components": sub_components_list
    }

def export_layout_to_json(
    layout_id: int,
    seed_used: int, # << 新增參數
    root_component: Component,
    gap_components: List[Component],
    final_leaf_components: List[Component],
    edges: List[Tuple[Tuple[float, float], Tuple[float, float]]],
    output_path: str
):
    """
    將完整的佈局資料（包含使用的種子）匯出成一個 JSON 檔案。
    """
    root_dict = component_to_dict(root_component)
    gap_dicts = [component_to_dict(comp) for comp in gap_components]
    leaf_dicts = [component_to_dict(comp) for comp in final_leaf_components]
    
    layout_data = {
        "layout_id": layout_id,
        "seed_used": seed_used, # << 新增欄位
        "root_component": root_dict,
        "gap_components": gap_dicts,
        "final_leaf_components": leaf_dicts,
        "netlist_edges": edges
    }

    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(layout_data, f, indent=4)
        print(f"📄 佈局資料已成功儲存至 {output_path}")
    except Exception as e:
        print(f"❌ 儲存 JSON 檔案至 {output_path} 時發生錯誤: {e}")

In [ ]:
def main_execution_batch_from_yaml():
    """
    實現最終的種子管理策略：
    1. 每次執行都會產生隨機且不同的結果。
    2. 為批次中的每個佈局使用獨立的隨機種子。
    3. 將使用的種子記錄到對應的 JSON 檔案中以供未來重現。
    """
    # --- 0. 載入設定檔 ---
    config = load_yaml_config('config.yaml')
    if config is None:
        return
        
    path_config = config.get('path_settings', {})
    main_config = config.get('main_execution', {})
    
    # --- 輸出目錄與檔案設定 ---
    raw_output_dir = path_config.get('raw_output_directory', 'raw_layouts')
    num_to_generate = main_config.get('num_layouts_to_generate', 1)
    file_basename = os.path.basename(raw_output_dir)
    image_subdir = path_config.get('image_subdirectory', 'images')
    json_subdir = path_config.get('json_subdirectory', 'json_data')
    image_output_folder = os.path.join(raw_output_dir, image_subdir)
    json_output_folder = os.path.join(raw_output_dir, json_subdir)
    os.makedirs(image_output_folder, exist_ok=True)
    os.makedirs(json_output_folder, exist_ok=True)
    
    print(f"📂 圖片將儲存於: '{image_output_folder}'")
    print(f"📂 JSON 資料將儲存於: '{json_output_folder}'")
    print(f"🚀 批次產生任務啟動，預計產生 {num_to_generate} 套資料...")
    print("-" * 50)

    # --- 迴圈開始 ---
    for i in range(num_to_generate):
        # --- << 關鍵修改 >> 為本次迴圈產生一個全新的、獨立的隨機種子 ---
        current_seed = random.randint(0, 2**32 - 1)
        random.seed(current_seed)
        np.random.seed(current_seed)
        
        print(f"=============== 正在產生資料組 #{i+1}/{num_to_generate} (Seed: {current_seed}) ===============")

        # (後續流程完全不變)
        # --- 1. 初始化產生器 ---
        level_0_generator = Level_0(**config.get('Level_0', {}))
        level_1_generator = Level_1(**config.get('Level_1', {}))
        level_2_generator = Level_2(**config.get('Level_2', {}))
        gap_filler = GapFiller(**config.get('GapFiller', {}))
        netlist_generator = NetlistGenerator(**config.get('NetlistGenerator', {}))

        # --- 2. 產生階層式佈局 ---
        root_components = level_0_generator.generate()
        root_component = root_components[0]
        level_1_components = level_1_generator.generate(root_components)
        level_2_components = level_2_generator.generate(level_1_components, root_component)
        
        # --- 3. 執行 Gap Filling ---
        num_gaps_to_fill = main_config.get('num_gaps_to_fill', 0)
        gap_filler_threshold = main_config.get('gap_filler_activation_threshold', 0.2)
        gap_components = []
        occupied_area = sum(c.width * c.height for c in level_2_components)
        total_area = root_component.width * root_component.height
        if total_area > 0 and (total_area - occupied_area) / total_area > gap_filler_threshold:
            gap_components = gap_filler.fill(level_2_components, root_component, num_gaps_to_fill)

        # --- 4. 產生 Netlist ---
        final_leaf_components = level_2_components + gap_components
        _, edges = netlist_generator.generate(final_leaf_components)

        # --- 5. 視覺化單一結果 ---
        plotter = ComponentPlotter()
        components_to_plot = root_components + gap_components
        current_title = f"{main_config.get('output_title', 'Layout')} #{i} (Seed: {current_seed})"
        png_filename = f"{file_basename}_{i}.png"
        png_output_path = os.path.join(image_output_folder, png_filename)
        plotter.plot(components_to_plot, title=current_title, edges=edges, output_filename=png_output_path)

        # --- 6. 匯出成 JSON 檔案 ---
        json_filename = f"{file_basename}_{i}.json"
        json_output_path = os.path.join(json_output_folder, json_filename)
        
        export_layout_to_json(
            layout_id=i,
            seed_used=current_seed, # 將本次使用的種子傳入
            root_component=root_component,
            gap_components=gap_components,
            final_leaf_components=final_leaf_components,
            edges=edges,
            output_path=json_output_path
        )
        print("-" * 50)

    print(f"✨ 所有批次任務執行完畢！ ✨")


# 執行使用 YAML 設定檔的批次產生流程
main_execution_batch_from_yaml()

🌱 使用隨機的主種子: 1756453849 (每次執行批次的結果將不同)
📂 圖片將儲存於: 'raw_layouts\images'
📂 JSON 資料將儲存於: 'raw_layouts\json_data'
🚀 批次產生任務啟動，預計產生 5 套資料...
--------------------------------------------------
=============== 正在產生資料組 #1/5 (Seed: 1756453849) ===============
[*] 開始為 15 個元件產生 Netlist...
[*] 初始機率性產生了 4 條邊。
[*] 發現 12 個獨立的元件群，開始橋接...
[*] Netlist 產生完畢，最終總共有 15 條邊。
[*] 正在繪製 15 條邊...
[*] 正在繪製 27 個已連接的引腳...
✅ 繪圖完成！圖片已儲存至 raw_layouts\images\raw_layouts_0.png
📄 佈局資料已成功儲存至 raw_layouts\json_data\raw_layouts_0.json
--------------------------------------------------
=============== 正在產生資料組 #2/5 (Seed: 1756453850) ===============
[*] 開始為 5 個元件產生 Netlist...
[*] 初始機率性產生了 0 條邊。
[*] 發現 5 個獨立的元件群，開始橋接...
[*] Netlist 產生完畢，最終總共有 4 條邊。
[*] 正在繪製 4 條邊...
[*] 正在繪製 7 個已連接的引腳...
✅ 繪圖完成！圖片已儲存至 raw_layouts\images\raw_layouts_1.png
📄 佈局資料已成功儲存至 raw_layouts\json_data\raw_layouts_1.json
--------------------------------------------------
=============== 正在產生資料組 #3/5 (Seed: 1756453851) ===============
[*] 開始為 3 個元件產生 Netlist...
[*